# Recommender System

In [1]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt

# Import datasets
- $X$: $num_{movies}$ (1682)  $\times$ $num_{features}$ (10) matrix of **movie features.**
- $\Theta$: $num_{users}$ (943)  $\times$ $num_{features}$ (10) matrix of **user features.**
- $Y$ : $num_{movies}$ $\times$ $num_{users}$ matrix of user ratings of movies  .
- $R$ : $num_{movies}$ $\times$ $num_{users}$ matrix, where $R(i, j) = 1$ if the $i^{th}$ movie was rated by the $j^{th}$ user.

In [2]:
movies_mat = sio.loadmat("./ex8_movies.mat")
params_mat = sio.loadmat("./ex8_movieParams.mat")

X, Theta = params_mat["X"], params_mat["Theta"]
Y, R = movies_mat["Y"], movies_mat["R"]

X.shape, Theta.shape, Y.shape, R.shape

((1682, 10), (943, 10), (1682, 943), (1682, 943))

# Serialize and de-serialize

In [3]:
def serialize(X, Theta):
    return np.concatenate((X.ravel(), Theta.ravel()))

def deserialize(param_seq, n_mov, n_fea, n_usr):
    X = param_seq[:n_mov * n_fea].reshape(n_mov, n_fea)
    Theta = param_seq[n_mov * n_fea:].reshape(n_usr, n_fea)
    return X, Theta

# Gradient descent
## Cost function

To perform collaborative filtering, we combine the $J(\Theta)$ and $J(X)$ cost funtions into one:
$$
\begin{aligned}
&J\left(x^{(1)}, \ldots, x^{\left(n_{m}\right)}, \theta^{(1)}, \ldots, \theta^{\left(n_{u}\right)}\right)
=\frac{1}{2} \sum_{(i, j): r(i, j)=1}\left(\left(\theta^{(j)}\right)^{T} x^{(i)}-y^{(i, j)}\right)^{2}+\frac{\lambda}{2} \sum_{i=1}^{n_{m}} \sum_{k=1}^{n}\left(x_{k}^{(i)}\right)^{2}+\frac{\lambda}{2} \sum_{j=1}^{n_{u}} \sum_{k=1}^{n}\left(\theta_{k}^{(j)}\right)^{2}
\end{aligned}
$$

## Gradient of cost funtion:
We can synchronously update $X$ and $\Theta$:
$$
Repeat \{ \\
\begin{array}{c}
x_{k}^{(i)}:=x_{k}^{(i)}-\alpha\left(\sum_{j: r(i, j)=1}\left(\left((\theta^{(j)}\right)^{T} x^{(i)}-y^{(i, j)}) \theta_{k}^{j}+\lambda x_{k}^{(i)}\right)\right. \\
\theta_{k}^{(i)}:=\theta_{k}^{(i)}-\alpha\left(\sum_{i: r(i, j)=1}\left(\left((\theta^{(i)}\right)^{T} x^{(i)}-y^{(i, j)}) x_{k}^{(i)}+\lambda \theta_{k}^{(j)}\right)\right.
\end{array} \\
\}
$$

In [4]:
def cost(param_seq, Y, R, n_fea, learning_rate=1):
    n_mov, n_usr = Y.shape
    
    X, Theta = deserialize(param_seq, n_mov, n_fea, n_usr)
    
    # Dot multplication with R:
    # 1 -> has rate -> keep value
    # 0 -> no rate -> val * 0 = 0
    
    inner = np.multiply((X @ Theta.T - Y), R)
    cost_term = 0.5 * np.power(inner, 2).sum()
    
    reg_term = 0.5 * learning_rate * np.power(param_seq, 2).sum()
    
    return cost_term + reg_term

def gradient(param_seq, Y, R, n_fea, learning_rate=1):
    n_mov, n_usr = Y.shape
    
    X, Theta = deserialize(param_seq, n_mov, n_fea, n_usr)
    
    inner = np.multiply((X @ Theta.T - Y), R)
    
    X_grad = inner @ Theta
    Theta_grad = inner.T @ X
    
    # Element-wise addition: each grad plus the corresponding regularize term gradient
    return serialize(X_grad, Theta_grad) + learning_rate * param_seq

# Add a new user
We can add a new user with some rating on some movies. Insert the new user to column 0.

In [5]:
# Y matrix column
usr_rating = np.zeros(Y.shape[0])

# rate some movies (0 - 5), not have to rate all movies as we can use model to predict
usr_rating[0] = 4
usr_rating[6] = 3
usr_rating[11] = 5
usr_rating[53] = 4
usr_rating[63] = 5
usr_rating[65] = 3
usr_rating[68] = 5
usr_rating[97] = 2
usr_rating[182] = 4
usr_rating[225] = 5
usr_rating[354] = 5

# R matrix column
usr_r = (usr_rating != 0).astype(int)

usr_rating.shape, usr_r.shape

((1682,), (1682,))

In [6]:
Y = np.insert(Y, 0, usr_rating, axis=1)
R = np.insert(R, 0, usr_r, axis=1)

Y.shape, R.shape

((1682, 944), (1682, 944))

# Random initialize $X$ and $\Theta$

In [7]:
eps = 0.12
param_seq = np.random.uniform(-eps, eps, X.shape[0] * X.shape[1] + Y.shape[1] * Theta.shape[1])

# Mean normalization $Y$
If a user has not rated any movie, then we assign it with mean of the rating.

In [8]:
mu = Y.mean(axis=1).reshape(-1,1)
Y_norm = Y - mu
Y_norm.shape, mu.shape

((1682, 944), (1682, 1))

# Training model

In [9]:
import scipy.optimize as opt

res = opt.minimize(fun = cost, 
                   x0=param_seq, 
                   args=(Y_norm, R, X.shape[1], 1),
                   method="TNC",
                   jac=gradient)

In [10]:
X_trained, Theta_trained = deserialize(res.x, X.shape[0], X.shape[1], Y.shape[1])
X_trained.shape, Theta_trained.shape

((1682, 10), (944, 10))

In [11]:
cost(res.x, Y_norm, R, X.shape[1], 1)

29535.414108825575

In [25]:
Y_pred = X_trained @ Theta_trained.T # predict Y for all user
new_usr_pred = Y_pred[:, 0] + mu.reshape(-1,)
rate_index = np.argsort(new_usr_pred)[::-1]
rate_index

array([  49,  180,  171, ...,  905,  990, 1535], dtype=int64)

In [26]:
new_usr_pred[rate_index][:10]

array([5.67914263, 5.62692338, 5.26100146, 5.17544055, 5.03088895,
       4.967269  , 4.78738471, 4.72128774, 4.71823973, 4.67467707])

In [27]:
movie_list = []

with open('./movie_ids.txt', encoding='latin-1') as f:
    for line in f:
        tokens = line.strip().split(' ')
        movie_list.append(' '.join(tokens[1:]))

movie_list = np.array(movie_list)

In [28]:
for m in movie_list[rate_index][:10]:
    print(m)

Star Wars (1977)
Return of the Jedi (1983)
Empire Strikes Back, The (1980)
Raiders of the Lost Ark (1981)
Independence Day (ID4) (1996)
Renaissance Man (1994)
Titanic (1997)
G.I. Jane (1997)
With Honors (1994)
Don't Be a Menace to South Central While Drinking Your Juice in the Hood (1996)
